In [2]:
import pandas as pd
from datetime import datetime, timezone
from scipy import stats
import matplotlib.pyplot as plt
import requests, json
import numpy as np
from collections import defaultdict 
import os

In [10]:
@np.vectorize
def to_categorical(data):
    response = {
        "N":0, "NE":1, "E":2, "SE":3, "S":4, "SW":5, "W":6, "NW":7,
    }
    return response.get(data)

def degrees_to_radians_adjusted(degrees):
    return (180 - degrees) * np.pi / 180

@np.vectorize
def degree2directions(data):
    threasholds = {
        (0, 22.5): "N",
        (22.5, 67.5): "NE",
        (67.5, 112.5): "E",
        (112.5, 157.5): "SE",
        (157.5, 202.5): "S",
        (202.5, 247.5): "SW",
        (247.5, 292.5): "W",
        (292.5, 337.5): "NW",
        (337.5, 360): "N"
    }
    for threashold, direction in threasholds.items():
        if threashold[0] <= data < threashold[1]:
            return direction
    
def read_vazao():
    df_vazao = pd.read_excel('../../dados/20240507_vazao_enchente.xlsx')
    df_dict = defaultdict(list)
    for i, df_sel in df_vazao.iterrows():
        time = datetime.strptime(f"{df_sel.data}T{df_sel.hora_utc}", "%d/%m/%YT%H:%M:%S")
        df_dict['times'].append(time)
        df_dict['vazao_m3s-1_bt'].append(df_sel['vazao_m3s-1_bt'])
        #df_dict['v_max_ms-1_bt'].append(df_sel['v_max_ms-1_bt'])
        #df_dict['v_media_ms-1_bt'].append(df_sel['v_media_ms-1_bt'])    
    df_vazao = pd.DataFrame(df_dict)
    df_vazao.index = [pd.to_datetime(t).tz_localize('UTC') for t in df_vazao.times]
    df_vazao = df_vazao.drop(columns=['times'])
    return df_vazao

    

In [11]:
def read_level(t0, t1, _id=303, params="water_l1,avg_water_l1,water_l1_ibge,avg_water_l1_ibge,water_l1_dhn,avg_water_l1_dhn"):
    fmt = "%Y-%m-%dT%H:%M:%S"
    tref = datetime.strptime('1970-01-01T00', "%Y-%m-%dT%H").replace(tzinfo=timezone.utc)
    t0, t1 = datetime.strptime(t0, fmt).replace(tzinfo=timezone.utc), \
                datetime.strptime(t1, fmt).replace(tzinfo=timezone.utc)
    print(f'requesting data from {t0} to {t1}')
    url = "https://simcosta.furg.br/api/intrans_data?" \
            f"boiaID={_id}&type=json&time1={(t0 - tref).total_seconds()}&time2={(t1 - tref).total_seconds()}" \
            f"&params={params}"
    response_level = requests.get(url).json()
    parse_response_level = [
        ({
            'timestamp': [r['timestamp']], \
            'YEAR': [r['YEAR']], \
            'MONTH': [r['MONTH']], \
            'DAY': [r['DAY']], \
            'HOUR': [r['HOUR']], \
            'MINUTE': [r['MINUTE']], \
            'SECOND': [r['SECOND']], \
            'water_l1': [r['water_l1']], \
            'water_l1_ibge': [r['water_l1_ibge']], \
            'water_l1_dhn': [r['water_l1_dhn']], \
            'avg_water_l1': [r['avg_water_l1']], \
            'avg_water_l1_ibge': [r['avg_water_l1_ibge']], \
            'avg_water_l1_dhn': [r['avg_water_l1_dhn']]
        }) for r in response_level
    ]
    df_level = pd.concat([pd.DataFrame(p) for p in parse_response_level])
    df_level = df_level.drop(columns=['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'SECOND'])
    for i, col in enumerate(df_level.columns):
        if i > 0:
            df_level.loc[:, col] = df_level[col].astype('float')
    df_level.index = pd.to_datetime(df_level['timestamp'])
    df_level = df_level.drop(columns=['timestamp']).dropna()
    return df_level

In [12]:
def read_wind(
    t0, t1, _id=303, 
    params="wind_speed,Avg_Wnd_Sp,wind_direction_n,Avg_Wnd_Dir_N"
):
    fmt = "%Y-%m-%dT%H:%M:%S"
    tref = datetime.strptime('1970-01-01T00', "%Y-%m-%dT%H").replace(tzinfo=timezone.utc)
    t0, t1 = datetime.strptime(t0, fmt).replace(tzinfo=timezone.utc), \
                datetime.strptime(t1, fmt).replace(tzinfo=timezone.utc)
    print(f'requesting data from {t0} to {t1}')
    url = "https://simcosta.furg.br/api/intrans_data?" \
            f"boiaID={_id}&type=json&time1={(t0 - tref).total_seconds()}&time2={(t1 - tref).total_seconds()}" \
            f"&params={params}"
    
    response = requests.get(url).json()
    parse_response = [
        ({
            'timestamp': [r['timestamp']], \
            'YEAR': [r['YEAR']], \
            'MONTH': [r['MONTH']], \
            'DAY': [r['DAY']], \
            'HOUR': [r['HOUR']], \
            'MINUTE': [r['MINUTE']], \
            'SECOND': [r['SECOND']], \
            'wind_speed': [r['wind_speed']], \
            'Avg_Wnd_Sp': [r['Avg_Wnd_Sp']], \
            'wind_direction_N': [r['wind_direction_N']], \
        }) for r in response
    ]
    df_wind = pd.concat([pd.DataFrame(p) for p in parse_response])
    df_wind = df_wind.drop(columns=['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'SECOND'])
    for i, col in enumerate(df_wind.columns):
        if i > 0:
            df_wind.loc[:, col] = df_wind[col].astype('float')
    df_wind.index = pd.to_datetime(df_wind['timestamp'])
    df_wind = df_wind.drop(columns=['timestamp']).dropna()
    return df_wind

In [13]:
def get_lags(variable, ref_var):
    laggeds = list()
    for lag in np.arange(-72, 1, 1):
        laggeds.append(pd.DataFrame({f'{str(lag).zfill(4)}':df[variable].shift(lag).values}))

    laggeds = pd.concat(laggeds, axis=1)
    laggeds = laggeds.assign(ref_var=df[ref_var].values).dropna()

    corrs = [
        np.corrcoef(laggeds[str(l)].values, laggeds['ref_var'])[0][1] \
        for i, l in enumerate(list(laggeds.columns)) if l != 'ref_var'
    ]
    return laggeds, corrs

In [14]:
def plot_lag_correlation(ref_var='water_l1_ibge',variables=['wind_speed', 'wind_direction_N']):
    n_axs = len(variables)
    fig, axs = plt.subplots(n_axs, 1, figsize=(30,20), sharey=True, sharex=True)
    for i, variable in enumerate(variables):
        laggeds, corrs = get_lags(variable, ref_var)
        xticks = [int(i) for i in list(laggeds.columns) if i != 'ref_var']
        
        for j, c in enumerate(corrs):
            if abs(c) >= 0.5:
                axs[i].scatter(xticks[j], c, color='black')
            
        axs[i].plot(xticks, corrs)
        for a in ['x', 'y']:
            axs[i].tick_params(axis=a, labelsize=25)
        axs[i].set_ylabel(f'{variable} correlation',fontsize=35)
        axs[i].set_xlabel('lags',fontsize=35)
        axs[i].set_title(f'{ref_var} x {variable}', fontsize=50)
        axs[i].grid()
    
    plt.tight_layout()
    plt.yticks(np.arange(-1,1+0.2,0.2), fontsize=35)
    plt.xticks(np.arange(-72,1,1), rotation=45, fontsize=35)
    path_out = "figuras/2024051300/{name}.png"
    os.makedirs(os.path.dirname(path_out.format(name='lag_corr')), exist_ok=True)
    plt.savefig(path_out.format(name='lagg_corr'))
    print(path_out.format(name='speed'))
    plt.legend()
    plt.show()
    
    
#variables = ["wind_speed","class_wind_direction_N"]
 #"Avg_Wnd_Sp", 
#refvars = ["water_l1_ibge"]
#["water_l1", "water_l1_ibge", "water_l1_dhn", "avg_water_l1", "avg_water_l1_ibge", "avg_water_l1_dhn"]
#for refvar in refvars:
#    print(variables)
#    plot_lag_correlation(refvar, variables)

In [15]:
def plot_boxplot(df):
    print('01 - *'*10)    
    #df[['vazao_m3s-1_bt']].boxplot(figsize=(20,8))
    #plt.xticks(rotation=45,fontsize=20);
    #path_out = "figuras/2024051300/{name}.png"
    #os.makedirs(os.path.dirname(path_out.format(name='vazao')), exist_ok=True)
    #plt.savefig(path_out.format(name='vazao'))
    #print(path_out.format(name='vazao'))
    #plt.show()
    
    print('02 - *'*10)
    df[['wind_direction_N']].boxplot(figsize=(20,8))
    plt.xticks(fontsize=20, rotation=50);
    #[i for i in range(len(drctions))]
    path_out = "figuras/2024051300/{name}.png"
    os.makedirs(os.path.dirname(path_out.format(name='direction')), exist_ok=True)
    plt.savefig(path_out.format(name='direction'))
    print(path_out.format(name='direction'))
    plt.show()

    print('03 - *'*10)
    df[['water_l1', 'water_l1_ibge', 'water_l1_dhn',
         'avg_water_l1_dhn', 'avg_water_l1_ibge']].boxplot(figsize=(20,8))
    #.drop(columns=['wind_direction_N','vazao_m3s-1_bt'])
    plt.xticks(rotation=45,fontsize=20);
    path_out = "figuras/2024051300/{name}.png"
    os.makedirs(os.path.dirname(path_out.format(name='level')), exist_ok=True)
    plt.savefig(path_out.format(name='level'))
    print(path_out.format(name='level'))
    plt.show()
    
    print('04 - *'*10)
    df[['wind_speed', 'Avg_Wnd_Sp']].boxplot(figsize=(20,8))
    #.drop(columns=['wind_direction_N','vazao_m3s-1_bt'])
    plt.xticks(rotation=45,fontsize=20);
    path_out = "figuras/2024051300/{name}.png"
    os.makedirs(os.path.dirname(path_out.format(name='speed')), exist_ok=True)
    plt.savefig(path_out.format(name='speed'))
    print(path_out.format(name='speed'))
    plt.show()

In [16]:
def aggregate_and_convert(df_raw):
    df = df_raw.resample('1h').mean()
    #df['class_wind_direction_N'] = degree2directions(df['wind_direction_N'])

    for v in ['wind_speed', 'Avg_Wnd_Sp']:
        df.loc[:, v] = df[v] * 3.6

    for v in ['water_l1', 'water_l1_ibge', 'water_l1_dhn', 
              'avg_water_l1', 'avg_water_l1_ibge', 'avg_water_l1_dhn']:
        df.loc[:, v] = df[v] / 100
    return df

In [17]:
df_vazao = read_vazao()
df_level = read_level("2024-05-01T00:00:00", datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))
df_wind = read_wind("2024-05-01T00:00:00", datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))

requesting data from 2024-05-01 00:00:00+00:00 to 2024-05-13 16:58:51+00:00


/tmp/ipykernel_73543/1533892018.py:32: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_level.loc[:, col] = df_level[col].astype('float')


requesting data from 2024-05-01 00:00:00+00:00 to 2024-05-13 16:59:07+00:00


/tmp/ipykernel_73543/1720065068.py:33: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_wind.loc[:, col] = df_wind[col].astype('float')


In [18]:
df_raw = pd.concat([df_wind, df_level, df_vazao], axis=1).astype('float') #
df_raw.to_csv('data_raw.csv')

In [19]:
df = aggregate_and_convert(df_raw)
df.to_csv('data_hourly.csv')

In [ ]:
df['class_wind_direction_N'] = df['wind_direction_N'].map(degree2directions)
df['class_wind_direction_N']

In [ ]:
plot_boxplot(df)
for i, v in enumerate(['wind_speed', 'Avg_Wnd_Sp']):
    df.loc[:, v][df[v]>100] = np.nan

print('05 - *'*10)
df[['wind_speed', 'Avg_Wnd_Sp']].boxplot(figsize=(20,8))
#.drop(columns=['wind_direction_N','vazao_m3s-1_bt'])
plt.xticks(rotation=45,fontsize=20);
plt.show()

# Analise da correlação de velocidade:
### 1. Correlação negativa durante as primeiras 10 horas
### 2. A partir de algumas horas, 36 neste caso, houve correlação positiva

# Analise da correlação de direção:
### 1. Difícil interpretação

In [ ]:
variables = ["wind_speed","wind_direction_N"]
 #"Avg_Wnd_Sp", 
refvars = ["water_l1_ibge"]
#["water_l1", "water_l1_ibge", "water_l1_dhn", "avg_water_l1", "avg_water_l1_ibge", "avg_water_l1_dhn"]
for refvar in refvars:
    plot_lag_correlation(refvar, variables)

In [ ]:
variables = ["wind_speed","wind_direction_N"]
refvars = ["water_l1_ibge"]
laggeds, corrs = get_lags('wind_speed', 'water_l1_ibge')
i_corrs = [(i, value) for i, value in enumerate(np.where(np.abs(corrs)>=0.5, corrs, None)) if value is not None]
corr_laggeds = laggeds.iloc[:,[i for i,v in i_corrs]]

In [ ]:
init = '20240509T00'
df_sel = df[df.index >= init]
index = [t.to_pydatetime() for t in df_sel.index ] #strftime('%Y%m%dT%H')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8), dpi=300)

ax.plot(index, df_sel['avg_water_l1_ibge'].values, label='Nível do Canal Boia (m)', color='blue')

ax2 = ax.twinx()
ax2.plot(index, df_sel['wind_speed'].values, label='Velocidade do Vento', color='black')

for i, l in enumerate(list(corr_laggeds.columns)):
    ax2.plot(df_sel['wind_speed'].shift(int(l)), '--', label=l, alpha=0.5)

ax.grid(True, linestyle='--', linewidth=0.5, color='gray')
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
ax.set_title('Dados de Nível do Canal Boia, Velocidade do Vento, Direção do Vento')
ax.set_xlabel('Data/Hora')
ax.set_ylabel('Nível do Canal Boia (m) / Maré')
ax2.set_ylabel('Velocidade do Vento (km/h)')

for i, direcao in enumerate(df_sel['wind_direction_N']):
    direcao = degrees_to_radians_adjusted(direcao)
    wind_x = np.sin(direcao)
    wind_y = np.cos(direcao)
    ax2.quiver(index[i], df_sel['wind_speed'].values[i], wind_x, wind_y, scale=100)

#for i, direcao in enumerate(degree2directions(df_sel['wind_direction_N'])):
    #vazao = df_sel['vazao_m3s-1_bt'][i]
    #if vazao > 0:
        #ax2.scatter(df_sel.index[i], 1.5, color='black');
        #ax2.text(df_sel.index[i], 1.5, int(vazao), ha='center', fontsize=12, rotation=45);
#    ax2.text(index[i], df_sel['wind_speed'].values[i], direcao, ha='center', fontsize=12, rotation=45);


plt.xticks(rotation=45);

plt.tight_layout()
plt.show()

In [ ]:

def degrees_to_radians_adjusted(degrees):
    return (180 - degrees) * np.pi / 180


# Adicionando o texto com a direção do vento
for i, direcao in enumerate(df_sel['wind_direction_N']):
    direcao = degrees_to_radians_adjusted(direcao)
    wind_x = np.sin(direcao)
    wind_y = np.cos(direcao)
    ax2.quiver(index[i], index[i], wind_x, wind_y, scale=50)
    #ax2.text(index[i], df_sel['wind_speed'].values[i], direcao, ha='center', fontsize=12, rotation=45);



In [ ]:
sel_lags['-063'].plot()